In [10]:
import pandas as pd
import numpy as np

# 1. load data
# read rawdata.xlsx
file_path = 'rawdata.xlsx'
adni_org_df = pd.read_excel(file_path, sheet_name='ADNI Org.')
csf_biomarker_df = pd.read_excel(file_path, sheet_name='CSF Biomarker')

# initialize an empty dataframe
df = pd.DataFrame(columns=['RID', 'EXAMDATE', 'AGE', 'ABETA', 'TAU', 'N', 'C'])

# processing 'ADNI Org.' sheet to get RID, EXAMDATE, AGE, C, N
for index, row in adni_org_df.iterrows():
    rid = row['RID']
    
    # check if there is this RID in 'CSF Biomarker'
    if rid in csf_biomarker_df['RID'].values:
        # load 'EXAMDATE', 'AGE', 'C', 'N' in 'ADNI Org.'
        examdate = row['EXAMDATE']
        age = row['AGE']
        c = row['C']
        n = row['N']
        
        # combining into result_df
        new_row = pd.DataFrame({
            'RID': [rid],
            'EXAMDATE': [examdate],
            'AGE': [age],
            'C': [c],
            'N': [n],
            'ABETA': [None],
            'TAU': [None],
        })
        df = pd.concat([df, new_row], ignore_index=True)

# processing 'CSF Biomarker'
for index, row in csf_biomarker_df.iterrows():
    rid = row['RID']
    drwdte = row['DRWDTE']
    
    # check if there is this RID in 'ADNI Org.'
    if rid in adni_org_df['RID'].values:
        # check if there  is same RID and DRWDTE in result_df
        match = df[(df['RID'] == rid) & (df['EXAMDATE'] == drwdte)]
        
        if not match.empty:
            # if true，upload 'ABETA' and 'TAU'
            df.loc[match.index, 'ABETA'] = row['ABETA']
            df.loc[match.index, 'TAU'] = row['TAU']
        else:
            # if false, create a new row
            new_row = pd.DataFrame({
                'RID': [rid],
                'EXAMDATE': [drwdte],
                'AGE': [None],
                'C': [None],
                'N': [None],
                'ABETA': [row['ABETA']],
                'TAU': [row['TAU']]
            })
            df = pd.concat([df, new_row], ignore_index=True)

# Sorting
df = df[['RID', 'EXAMDATE', 'AGE', 'ABETA', 'TAU', 'N', 'C']]

# delete rows whose ABETA, TAU, C, N are all empty or 0
condition = (df[['ABETA', 'TAU', 'C', 'N']].isna() | (df[['ABETA', 'TAU', 'C', 'N']] == 0)).all(axis=1)
df = df[~condition]

# 2.  recalculate the age for each RID, as it's the age at baseline in the document
grouped = df.groupby('RID')
updated_rows = []

for rid, group in grouped:
    group = group.sort_values(by='EXAMDATE')
    
    # get the first EXAMDATE and AGE for the group
    first_age = group['AGE'].iloc[0]
    
    # calculating the AGE for the rest of the rows
    for i, row in group.iterrows():
        if i == group.index[0]:
            # The first line remains the original AGE
            updated_rows.append(row)
        else:
            # calculate the new AGE based on the first EXAMDATE and AGE, keeping one decimal place
            date_diff = (row['EXAMDATE'] - group['EXAMDATE'].iloc[0]).days / 365
            new_age = round(first_age + date_diff, 1)
            row['AGE'] = new_age
            updated_rows.append(row)

# reassembly
df = pd.DataFrame(updated_rows)

# delete the rows whose EXAMDATE is empty
df = df.replace(0, np.nan)
df = df.dropna(subset=['EXAMDATE'])

# 3. combining the rows with same RID and AGE
grouped = df.groupby(['RID', 'AGE'])
# initialize an empty list to store the processed rows
merged_rows = []

for (rid, age), group in grouped:
    group = group.sort_values(by='AGE')
    merged_row = {
        'RID': rid,
        'AGE': group['AGE'].iloc[0],
        'ABETA': np.nan,
        'TAU': np.nan,
        'N': np.nan,
        'C': np.nan,
    }
    
    for _, row in group.iterrows():
        if not pd.isna(row['ABETA']) or not pd.isna(row['TAU']):
            # if there is ABETA or TAU at this time point, fill them in
            merged_row['ABETA'] = row['ABETA']
            merged_row['TAU'] = row['TAU']
        if not pd.isna(row['C']) or not pd.isna(row['N']):
            # if there is C or N at this time point, fill them in
            merged_row['C'] = row['C']
            merged_row['N'] = row['N']

    merged_rows.append(merged_row)

df = pd.DataFrame(merged_rows)

In [11]:
# 4. 清洗与统计

# 4.1 删去 df 中任何列含 NaN 的行（只保留六个关键列都非空的记录）
df = df.dropna(subset=['RID', 'AGE', 'ABETA', 'TAU', 'N', 'C'])

# 4.2 删除仅有一行数据的 RID
rows_per_rid = df.groupby('RID').size()
valid_rids = rows_per_rid[rows_per_rid > 1].index
df = df[df['RID'].isin(valid_rids)].copy()

# 4.3 统计
rows_per_rid = df.groupby('RID').size()
total_rids = rows_per_rid.index.nunique()
rid_count_by_rows = rows_per_rid.value_counts().sort_index()  # 行数 -> 对应 RID 个数

print(f"Total unique RIDs after filtering: {total_rids}")
print("Counts of RIDs by number of rows:")
for num_rows, num_rids in rid_count_by_rows.items():
    print(f"{num_rows} rows: {num_rids} RIDs")

# 如需得到一个汇总表：
summary_df = rid_count_by_rows.rename_axis('num_rows').reset_index(name='num_RIDs')
print("\nSummary table:\n", summary_df)

Total unique RIDs after filtering: 251
Counts of RIDs by number of rows:
2 rows: 157 RIDs
3 rows: 58 RIDs
4 rows: 26 RIDs
5 rows: 10 RIDs

Summary table:
    num_rows  num_RIDs
0         2       157
1         3        58
2         4        26
3         5        10


In [12]:
# --- 5. 先生成原始数据表和统计表 ---
cols_to_stat = ['ABETA', 'TAU', 'N', 'C']
stats = df[cols_to_stat].agg(
    ['mean', 'std', lambda x: x.quantile(0.05), lambda x: x.quantile(0.95)]
)
stats.index = ['mean', 'std', 'y5', 'y95']

if "RID" in df.columns:
    df = df.rename(columns={"RID": "PID"})
rawdata = df.copy()

# --- 6. 再对 ABETA, TAU, N, C 做标准化 ---
cols_to_standardize = ['ABETA', 'TAU', 'N', 'C']
df[cols_to_standardize] = df[cols_to_standardize].apply(
    lambda x: (x - x.mean()) / x.std()
)                   

# --- 7. 保存到 data.xlsx ---
with pd.ExcelWriter("data.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, index=False)
    stats.to_excel(writer, sheet_name="stats")
    rawdata.to_excel(writer, sheet_name="rawdata")